In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from omegaconf import OmegaConf
from labeling.tools import psql_connect
import numpy as np

In [3]:
config = OmegaConf.load("config/etl.yaml")
cur = psql_connect(**config.dest).cursor()

## Validate

In [4]:
cur.execute(f"select distinct block_height from {config.dest.table} order by block_height;")
bh = cur.fetchall()
bh = np.array([b[0] for b in bh])

In [ ]:
bh.shape

In [ ]:
diff = np.diff(bh)
diff.sort()

from collections import Counter

c = Counter(diff)
c

## Block intervals

In [18]:
from labeling.upload import get_batch_intervals

vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
combine_win_size = config.hp.etl.combine_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size
intervals = get_batch_intervals(
    start_block=7063871,
    global_end_block=9291903,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
)

len(intervals)


62

In [20]:
b_size = 3
start = None

for idx, (bs, be) in enumerate(intervals):
    if start is None:
        start = bs
    if (idx + 1) % b_size == 0:
        print(start, be)
        start = None

7049472 7269070
7153872 7373470
7258272 7477870
7362672 7582270
7467072 7686670
7571472 7791070
7675872 7895470
7780272 7999870
7884672 8104270
7989072 8208670
8093472 8313070
8197872 8417470
8302272 8521870
8406672 8626270
8511072 8730670
8615472 8835070
8719872 8939470
8824272 9043870
8928672 9148270
9033072 9252670


In [23]:
for bs, be in get_batch_intervals(
    start_block=7049472,
    global_end_block=7269070,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be, be - bs + 1)

7049472 7199470 149999
7084272 7234270 149999
7119072 7269070 149999


In [ ]:
for bs, be in get_batch_intervals(
    start_block=7049472,
    global_end_block=7269070,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be, be - bs + 1)

## Missing block:
```
7272731 7272972 | 241
7883969 7884210 | 241
8699979 8700001 | 22
9057005 9057016 | 11
9057065 9057076 | 11
```

In [ ]:
diff = np.diff(bh)
combine_win_size = config.hp.etl.combine_win_size

for i, d in enumerate(diff):
    if d != combine_win_size:
        print(bh[i], bh[i+1], "|", d)

## Total block

In [5]:
cur.execute(f"select max(block_height) from public.validator_block;")
print(cur.fetchall())

[(9291903,)]


In [9]:
config.hp.etl

{'accept_rate': 0.1, 'concentration_level': 0.9, 'vote_score': 2, 'propose_score': 6, 'A': 9, 'B': 4, 'C': 2, 'D': 4, 'vote_proposed_win_size': 14400, 'combine_win_size': 150, 'label_win_size': 100800}

In [10]:
print("total blocks:", ((9070703 - 100800 - 150) - (7049472 + 14400 - 1)) / 150 + 1)

total blocks: 12706.88
